# HUHU@IberLEF2023 Task 1 (Binary Classification)

Task: https://sites.google.com/view/huhuatiberlef23/huhu

This notebook contains the code to load and test several trained transformers for the task of hurtful humour detection (binary classification).

In particular, the models are:

* BERT Multilingual: ``bert-base-multilingual-cased`` and ``bert-base-multilingual-uncased``
* RoBERTa: ``roberta-base``
* BETO: ``dccuchile/bert-base-spanish-wwm-cased`` and ``dccuchile/bert-base-spanish-wwm-uncased``
* DistilBERT Multilingual: ``distilbert-base-multilingual-cased``

The transformers that define the best ensemble found in experimentation will be used to predict the label of the instances in the test split.

The predicted labels will be stored in an output file as per the requirements of the competition. Additional evaluation plots may be also genered.

# Setting up the environment

In [1]:
import torch

# Check GPU availability on Google Colab
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

use_cuda = torch.cuda.is_available()

Fri May  5 20:22:49 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   58C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
# Install libraries
!pip install simpletransformers
!pip install datasets
!pip install ipywidgets
!pip install --upgrade huggingface_hub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.7/250.7 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 32.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 35.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 18.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━

In [3]:
# Define global variables

SEED = 42 # allow for experiments' reproductibility
WEIGHTED = True # use weighted ensemble (in favour of models with higher F1-score)

# Test split load

In [4]:
from huggingface_hub import notebook_login
# Notebook login via HF's token
notebook_login()

In [35]:
from datasets import *
import pandas as pd

# Avoid warnings
logging.set_verbosity_error()

# Load test split
test = pd.DataFrame(load_dataset("huhu2023/test-huhu2023", split='test'))

In [36]:
# Function to rename fields and drop unnecessary ones
def get_text_and_label(df, original_dataset=True):
  return df.rename(columns={"tweet": "text"})[["index", "text"]]

# Get treated dataframe for test split
test = get_text_and_label(test)

print("Test split size:", len(test.index))
test.head()

Test split size: 778


,index,text
0,52830,-Mamá en la escuela me dicen gorda -Pobresilla...
1,78883,"No te sientas diferente, da igual si eres negr..."
2,78926,Si esta asi.. SUPER SI.. y que se pongan celos...
3,61844,—Bebé ¿Me veo gorda con este vestido?\n—¡No mi...
4,78830,Las mujeres solo desean 2 cosas en la vida: co...


# Set-up the working environment

In [37]:
# Select the name of the experiment to be evaluated
EXP = "lr-2e-05-optimizer-Adafactor-epoch-10"
TRANSFORMERS = "top2_transformers"

In [38]:
# Load and mount the Drive helper
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [39]:
import os

# Define the path to the experiment folder
PATH = "/content/drive/My Drive/HUHU-IberLEF2023/bin/outputs/"
EXP_PATH = os.path.join(PATH, EXP)
TRANSFORMERS_PATH = os.path.join(PATH, TRANSFORMERS)
print("Current working dir:", EXP_PATH)

# Create a folder for the test results
OUTPUT = os.path.join(EXP_PATH, "test")
os.mkdir(OUTPUT)

Current working dir: /content/drive/My Drive/HUHU-IberLEF2023/bin/outputs/lr-2e-05-optimizer-Adafactor-epoch-10


# Models' load

In this section, the different transformers that will be tested are gathered. For this purpose, the implementation mainly relies in the ``simpletransformers`` Python library, which allows to train and test transformers within few steps.

For further information: https://simpletransformers.ai/

**IMPORTANT NOTE:** although this is a binary classification task, it will be treated as a regression where a value between 0 and 1 must be predicted for each instance. Later, these predictions will be turned into binary values by the corresponding ensemble.

In [42]:
# Define transformers' initialization dictionary 
models = {
    "mbert-cased": {
        "model_type": "bert",
        "model_name": "bert-base-multilingual-cased"
    },
    "mbert-uncased": {
        "model_type": "bert",
        "model_name": "bert-base-multilingual-uncased"
    },
    "roberta": {
        "model_type": "roberta",
        "model_name": "roberta-base"
    },
    "beto-cased": {
        "model_type": "bert",
        "model_name": "dccuchile/bert-base-spanish-wwm-cased"
    },
    "beto-uncased": {
        "model_type": "bert",
        "model_name": "dccuchile/bert-base-spanish-wwm-uncased"
    },
    "distilbert-multi": {
        "model_type": "distilbert",
        "model_name": "distilbert-base-multilingual-cased"
    }
}

In [43]:
# Import pre-trained simpletransformers models for classification
from simpletransformers.classification import ClassificationModel, ClassificationArgs

# Define a dictionary where each key matches its corresponding transformer
for model, fields in models.items():    
  models[model] = ClassificationModel(fields["model_type"], os.path.join(TRANSFORMERS_PATH, model))

# Best ensemble's definition

The ensemble that performed the best in the selected experiment is defined.

This will be the one used for the predictions to be performed on the test set.

In [44]:
import json

# Get the data relative to the best ensemble
best_ensemble = {}
with open(os.path.join(EXP_PATH, "best-ensemble.json")) as json_file:
    best_ensemble = json.load(json_file)

print("----- BEST ENSEMBLE -----")
for field in ["name", "models", "metrics"]:
  print(f"{field}:", best_ensemble.get(field))

----- BEST ENSEMBLE -----
name: ensemble62
models: ['mbert-cased', 'mbert-uncased', 'roberta', 'beto-cased', 'beto-uncased', 'distilbert-multi']
metrics: {'accuracy': 0.85794, 'f1': 0.77381, 'precision': 0.80247, 'recall': 0.74713}


# Best ensemble's predictions

In [45]:
# Load model evaluation JSON
model_evaluation = {}
for model in best_ensemble.get("models"):
  with open(os.path.join(os.path.join(EXP_PATH, model), "model-evaluation.json")) as json_file:
      model_evaluation[model] = json.load(json_file)

In [46]:
from sklearn.preprocessing import normalize

# Function which determines the ensembler prediction based on its
# transformers' predictions. A weighted voting system may be used
def vote(predictions, weighted=False, weights=None):
  voting = sum(predictions * weights) if weighted else sum(predictions)/len(predictions)
  return 0 if voting < 0.5 else 1

test_predictions = list()

# Function to predict the label of the instances in a dataset split (validation
# ("val") or test ("test")) for each ensemble
def predict_ensemble(ensemble_name, dataset_name, dataset, weighted=False):
  # Traverse each dataset instance
  for i in range(len(dataset.index)):
    predictions = list()
    ensemble_models = best_ensemble.get("models")
    # Get the raw output of each model in the ensemble for the instance at hand
    for model_name in ensemble_models:
      curr_model_outputs = model_evaluation[model_name].get(f"{dataset_name}_model_outputs")
      predictions.append(curr_model_outputs[i])
    
    # Define the list of weights if a weighted voting system must be used
    weights = list()
    if weighted:
      # The weights' list is obtained by normalizing the F1-scores of the models
      # in the ensemble
      f1_scores_list = [model_evaluation[model_name]["metrics"].get("f1")
                        for model_name in best_ensemble.get("models")]
      weights = normalize([f1_scores_list], norm="l1")[0]

    # Append the predicted label to the predictions of the ensemble
    ensemble_pred = vote(predictions, weighted, weights)
    test_predictions.append(ensemble_pred)

In [47]:
# Predicting the label of the test set's instances with each individual
# transformer that composes the best ensemble
for model_name in best_ensemble.get("models"):
  model_predictions, model_raw_outputs = models.get(model_name).predict(test["text"].tolist())
  model_evaluation[model_name]["test_model_outputs"] = model_raw_outputs
  model_evaluation[model_name]["test_predictions"] = model_predictions

# Calculating the test predictions of the best ensemble
predict_ensemble(best_ensemble.get("name"), "test", test, weighted=WEIGHTED)

# Show some predictions
n = 5
print(f"First {n} predictions:", test_predictions[:n])

  0%|          | 0/778 [00:00<?, ?it/s]

  0%|          | 0/98 [00:00<?, ?it/s]

  0%|          | 0/778 [00:00<?, ?it/s]

  0%|          | 0/98 [00:00<?, ?it/s]

  0%|          | 0/778 [00:00<?, ?it/s]

...

  0%|          | 0/778 [00:00<?, ?it/s]

  0%|          | 0/98 [00:00<?, ?it/s]

  0%|          | 0/778 [00:00<?, ?it/s]

  0%|          | 0/98 [00:00<?, ?it/s]

First 5 predictions: [1, 1, 0, 1, 1]


# Save test predictions

A new pandas dataframe is created. Further, the test predictions are saved to an output CSV file as per required by the competition.

In [48]:
# Create output dataframe for test predictions
test_output = test[["index"]].rename(columns={"index": "tweet_id"})
test_output["humour"] = test_predictions
test_output.head(10)

,tweet_id,humour
0,52830,1
1,78883,1
2,78926,0
3,61844,1
4,78830,1
5,48393,1
6,11875,1
7,62245,0
8,62467,1
9,40664,1


In [49]:
# Save results in CSV file
test_output.to_csv(os.path.join(OUTPUT, "results.csv"), index=False)